In [3]:
import ee
import geemap
import pprint

In [4]:
ee.Initialize()

## Choose the location and time

In [5]:
Map = geemap.Map(center=(37.5642, 127.0017), zoom=10)

In [6]:
# LC08_L1TP_116034_20180509_20180517_01_T1
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_116034_20180509')

## Load metadata of the “Landsat 8 collection 1 tier 1” image

In [7]:
pprint.pprint(image.getInfo())

{'bands': [{'crs': 'EPSG:32652',
            'crs_transform': [30, 0, 171585, 0, -30, 4269315],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7841, 7971],
            'id': 'B1'},
           {'crs': 'EPSG:32652',
            'crs_transform': [30, 0, 171585, 0, -30, 4269315],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7841, 7971],
            'id': 'B2'},
           {'crs': 'EPSG:32652',
            'crs_transform': [30, 0, 171585, 0, -30, 4269315],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7841, 7971],
            'id': 'B3'},
       

## Load “Landsat 8 collection 1 tier 1” Thermal Band image(B10) and Panchromatic band image(B8)

In [8]:
B10 = image.select('B10')
B8 = image.select('B8')

B10.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B10',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7841, 7971],
   'crs': 'EPSG:32652',
   'crs_transform': [30, 0, 171585, 0, -30, 4269315]}],
 'id': 'LANDSAT/LC08/C01/T1/LC08_116034_20180509',
 'version': 1527100853576144,
 'properties': {'RADIANCE_MULT_BAND_5': 0.006000100169330835,
  'RADIANCE_MULT_BAND_6': 0.0014921999536454678,
  'RADIANCE_MULT_BAND_3': 0.011626999825239182,
  'RADIANCE_MULT_BAND_4': 0.009804800152778625,
  'RADIANCE_MULT_BAND_1': 0.012322000227868557,
  'RADIANCE_MULT_BAND_2': 0.01261799968779087,
  'K2_CONSTANT_BAND_11': 1201.1441650390625,
  'K2_CONSTANT_BAND_10': 1321.078857421875,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[125.81040796607343, 38.52621353886707],
    [125.80921493863289, 38.523038890318425],
    [125.79013546663516, 38.45728699455111],
    [125.67237615389934, 38.04798764543425],
    [125.55576413239353, 37.6386080928

In [9]:
# vis_params_B10 = {'bands': ['B10'], 'min': 0.0, 'max': 28378.0, 'opacity': 1.0, 'gamma': 1.0}
# vis_params_B8 = {'bands': ['B8'], 'min': 0.0, 'max': 12772.0, 'opacity': 1.0, 'gamma': 1.0}
# Map.addLayer(B10, vis_params_B10,'B10')
# Map.addLayer(B8, vis_params_B8,'B8')
# Map

In [10]:
Gain_B10 = image.get('RADIANCE_MULT_BAND_10').getInfo()
Offset_B10 = image.get('RADIANCE_ADD_BAND_10').getInfo()
print(Gain_B10)
print(Offset_B10)

SR_B10 = (B10.multiply(Gain_B10)).add(Offset_B10)
SR_B10.getInfo()

0.00033420001273043454
0.10000000149011612


{'type': 'Image',
 'bands': [{'id': 'B10',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0.10000000149011612,
    'max': 22.001797835779143},
   'dimensions': [7841, 7971],
   'crs': 'EPSG:32652',
   'crs_transform': [30, 0, 171585, 0, -30, 4269315]}]}

In [11]:
K1_B10 = image.get('K1_CONSTANT_BAND_10').getInfo()
K2_B10 =image.get('K2_CONSTANT_BAND_10').getInfo()
print(K1_B10)
print(K2_B10)

774.8853149414062
1321.078857421875


In [12]:
BT_a = image.expression(
    '(K1 / SR_B10) + 1', {
        'K1': K1_B10,
        'SR_B10': SR_B10
    })
pprint.pprint(BT_a.getInfo())


# BT_b = BT_a.log()
# pprint.pprint(BT_b.getInfo())

BT = image.expression(
    '(K2 / BT_b) - 273.15', {
        'BT_b': BT_a.log(),
        'K2': K2_B10
    })

pprint.pprint(BT.getInfo())

# vis_params = {'bands': ['BT'], 'min': 0.0, 'max': 40.0, 'opacity': 1.0, 'gamma': 1.0}
# Map.addLayer(BT,{},'BT')
# Map

{'bands': [{'crs': 'EPSG:32652',
            'crs_transform': [30, 0, 171585, 0, -30, 4269315],
            'data_type': {'max': 7749.8530339471545,
                          'min': 36.21918166529528,
                          'precision': 'double',
                          'type': 'PixelType'},
            'dimensions': [7841, 7971],
            'id': 'constant'}],
 'type': 'Image'}
{'bands': [{'crs': 'EPSG:32652',
            'crs_transform': [30, 0, 171585, 0, -30, 4269315],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [7841, 7971],
            'id': 'constant'}],
 'type': 'Image'}


## Export

In [13]:
# #create function to download the image or imagecollection as you desire
# def downloader(ee_object,region): 
#     try:
#         #download image
#         if isinstance(ee_object, ee.image.Image):
#             print('Its Image')
#             url = ee_object.getDownloadUrl({
#                     'scale': 30,
#                     'crs': 'EPSG:4326',
#                     'region': region
#                 })
#             return url
        
#         #download imagecollection
#         elif isinstance(ee_object, ee.imagecollection.ImageCollection):
#             print('Its ImageCollection')
#             ee_object_new = ee_object.mosaic()
#             url = ee_object_new.getDownloadUrl({
#                     'scale': 30,
#                     'crs': 'EPSG:4326',
#                     'region': region
#                 })
#             return url
#     except:
#         print("Could not download")

In [14]:
# region = geometry.toGeoJSONString()#region must in JSON format

# path = downloader(MyImage,region)#call function

# print(path)#print the download URL

## Export to Google Drive (in Google Colab)

In [15]:
# # Export the image, specifying scale and region.
# task = ee.batch.Export.image.toDrive(**{
#   'image': BT,
#   'description': 'BT_Example',
#   'folder':'Example_folder',
#   'scale': 30
# })

# task.start()

In [16]:
# import time 
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)

## Download URL

In [24]:
geometry = ee.Geometry.Rectangle([127.0000, 37.5000, 127.3000, 37.5300])
region = geometry.toGeoJSONString()#region must in JSON format

url = BT.getDownloadUrl({
                    'scale': 30,
                    'region': region
                })

print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3243975a5b48b9bde60c845835ca8501-3e1859d78720893c4f2340d60507acea:getPixels
